In [92]:
!nvidia-smi

Thu Feb  1 21:53:12 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           On  | 00000000:06:00.0 Off |                    0 |
| N/A   39C    P0              60W / 300W |  12447MiB / 16384MiB |      1%   E. Process |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [103]:
%%writefile certify_gl.py
from datetime import datetime, date
import sys
import os
# os.environ['CUDA_VISIBLE_DEVICES']="2"
# os.environ['CUDA_LAUNCH_BLOCKING']="1"
import json
import yaml

import click
import scipy
import torch
from torch.utils.data import DataLoader, Dataset
import torchvision

from architectures import get_architecture
from datasets_utils import get_dataset, DATASETS, get_num_classes, get_normalize_layer
from src.numerical_methods import *
from src.certification_utils import *
from src.smoothing_and_attacks import *
from src.utils import *


def make_test_dataset(config):
    test_dataset = get_dataset(config["dataset"], "test")
    pin_memory = (config["dataset"] == "imagenet")
    np.random.seed(42)
    idxes = np.random.choice(len(test_dataset), config["NUM_IMAGES_FOR_TEST"], replace=False)
    
    ourdataset = make_our_dataset_v2(test_dataset, idxes)
    ourdataloader = DataLoader(ourdataset, shuffle=False, batch_size=1,
                         num_workers=6, pin_memory=False)
    return ourdataset, ourdataloader


def load_model(config):
    device = torch.device(config["device"])
    model = get_architecture(arch=config["arch"], dataset=config["dataset"], device=device)
    checkpoint = torch.load(config["base_classifier"], map_location=device)
    model.load_state_dict(checkpoint['state_dict'])
    model.eval()
    model = model.to(device)
    return model


def construct_bounds(ns, b_zero, x0, d, betas_list, type_of_transform, sigmas):
    shape = [b.shape[0] for b in betas_list]
    print("I'm here")
    shape = tuple(shape)
    betas = jnp.asarray(list(map(jnp.array, itertools.product(*betas_list))))
    sigma_b, sigma_c, sigma_tr, sigma_gamma, sigma_blur = sigmas
    gamma = construct_gamma(sigma_b=sigma_b, sigma_c=sigma_c, sigma_tr=sigma_tr, sigma_gamma=sigma_gamma, sigma_blur=sigma_blur)
    bounds, p, g = compute_normed_bounds(compute_bound, x0, gamma, b_zero, betas, key, ns, d, type_of_transform)
    x, xi = pxi_to_xi(p)
    z = csaps(betas_list, g.reshape(shape))
    
    hg = []

    for beta in tqdm(betas):
        hat_g = g_to_hat_g(z, beta, b_zero)
        hg.append(hat_g)

    hat_g = jnp.asarray(hg)

    hatg_int = csaps(betas_list, hat_g.reshape(shape)) #
    return xi, hatg_int

def do_log(filename, string):
    with open(filename, "a") as f:
        print(string, file=f, flush=True)
        

def calculate_general(config):
    
    # defining and loading main parameters and fucntions
    xi_tss = scipy.stats.norm.ppf
    
    device = torch.device(config["device"])
    sigmas = config["sigmas"]

    n0 = config["n0"]
    maxn = config["maxn"]
    adaptive = config["adaptive"]
    alpha = config["alpha"]
    bs = config["batch"]
    num_classes = config["num_classes"]
    
    
    b_zero = jnp.array(config["b_zero"])
    x0 = jnp.array(config["x0"])
    d = config["dimenshion"]
    type_of_transform = config["transform"]
    ns = config["ns"]
    
    # creating logfile and saving used config
    exp_start_time = "{:%Y_%m_%d_%H_%M_%S}".format(datetime.now())
    sigmas_values = "b_{}_c_{}_tr_{}_gamma_{}_blur_{}".format(*sigmas.values())
    exp_dir = os.path.join(config["log_dir"], type_of_transform, config["dataset"], sigmas_values, exp_start_time)
    if not os.path.exists(exp_dir):
        os.makedirs(exp_dir)
    filename = os.path.join(exp_dir, "res.txt")
    
    with open(os.path.join(exp_dir, "used_config.yaml"), "w") as f:
        yaml.dump(config, f)
        

    # loading base classifier f
    model = load_model(config)
    
    # creating test dataset to certify on
    ourdataset, ourdataloader = make_test_dataset(config)
    
    # constructing smoothing transfrom phi
    phi = construct_phi(config["transform"], device=device, **sigmas)
    
    
    # constructiong xi and hatg
    
    betas_dict = config["betas_estimation"]
    betas_list = []
    for i, key in enumerate(betas_dict):
        if i >= d:
            break
        beta = betas_dict[key]
        betas_list.append(jnp.linspace(beta["left"], beta["right"], beta["db"]))
        
    
    xi, hatg_int = construct_bounds(ns, b_zero, x0, d, betas_list, type_of_transform, sigmas.values())
    
    # defining and loading attack
    attack = construct_attack(type_of_transform)
    
    # calculating benign (vanilla) accuracy of base classifier f
    # accuracy of non-smoothed model on original images
    benign_acc = Accuracy(model, loader=ourdataloader, device=device)
    benign_acc = benign_acc.mean()
    string = f"Benign accuracy {benign_acc}"
    print(string)
    do_log(filename, string)
    
    
    # creating attack set B to certify model on
    betas_attack_dict = config["betas_certification"]
    betas_attack_list = []
    for i, key in enumerate(betas_attack_dict):
        if i >= d:
            break
        beta_attack = betas_attack_dict[key]
        betas_attack_list.append(np.linspace(beta_attack["left"], beta_attack["right"], beta_attack["db"]))
    betas_attack = np.asarray(list(map(np.array, itertools.product(*betas_attack_list))))

    # calculating statistics
    paCP, isOkCP = pa_isOk_collector(model, loader=ourdataloader, Phi=phi, device=device,
                           n0=n0, maxn=maxn, alpha=alpha, batch_size=bs, adaptive=adaptive,
                             num_classes=num_classes)
    res_dict = {}
    res_dict["pa"] = list(paCP)
    res_dict["is_ok"] = list(map(int, list(isOkCP)))
    with open(os.path.join(exp_dir, "res_dict.json"), 'w') as f:
        json.dump(res_dict, f, ensure_ascii=False)
    
    
    h_acc = np.mean(isOkCP)
    string = f"Ordinary accuracy of Smoothed Classiifer {h_acc}"
    print(string)
    do_log(filename, string)
    
    # calculate empirically robust accuracy of f
    f_era = None
    if config["calculate_f_era"]:
        f_era = ERA_Only_ND(model, ourdataloader, attack=attack, device=device, PSN=betas_attack)
        print(f"f_era {f_era}")
        do_log(filename, string)

    # calculate empirically robust accuracy of h (might be very time consuming)
    h_era = None
    if config["calculate_f_era"]:
        h_era = ERA_Only_For_Smoothed_ND(model, ourdataloader, attack, phi, device, 
                                 PSN=betas_attack, n0=n0, maxn=maxn, alpha=alpha, 
                                 batch_size=bs, adaptive=adaptive, num_classes=num_classes)
        print(f"h_era {h_era}")
        do_log(filename, string)

    # calulate our certified robust accuracy (CRA)
    hlist = np.linspace(config["hlist"]["left"], config["hlist"]["right"], config["hlist"]["n_steps"])
    print(hlist)
    hmin_ours = CertAccChecker(safe_beta, betas=betas_attack, hlist=hlist, xi=xi, hatg_int=hatg_int)
    print("HMIN!!!!!", hmin_ours)
    if hmin_ours:
        cert_acc_ours = ((paCP > hmin_ours).astype("int") * isOkCP).mean()
    else:
        cert_acc_ours = 0
        hmin_ours = None
    string = f"Cert Acc {type_of_transform} ours is {cert_acc_ours}.  h_min is {hmin_ours}"
    print(string)
    do_log(filename, string)


    # calculate TSS' CRA if applicable
    sb_tss = safe_beta_tss(type_of_transform, **sigmas)
    cert_acc_tss = None
    hmin_tss = None
    if sb_tss:
        hmin_tss = CertAccCheckerTSS(betas=betas_attack, hlist=hlist, xi=xi_tss, safe_beta_tss=sb_tss)
        if hmin_tss:
            cert_acc_tss = ((paCP > hmin_tss).astype("int") * isOkCP).mean()
        else:
            cert_acc_tss = 0
            hmin_tss = None
    string = f"Cert Acc {type_of_transform} TSS is {cert_acc_tss}.  h_min is {hmin_tss}"
    print(string)
    do_log(filename, string)


    # calculate MP's CRA if applicable
    cert_acc_mp = None
    hmin_mp = None
    if type_of_transform in ["c", "gamma"]:
        hmin_mp = CertAccCheckerTSS(betas=betas_attack, hlist=hlist, xi=None, safe_beta_tss=safe_beta_MP_gamma)
        if hmin_mp:
            cert_acc_mp = ((paCP > hmin_mp).astype("int") * isOkCP).mean()
        else:
            cert_acc_mp = 0
            hmin_mp = None
    string = f"Cert Acc {type_of_transform} MP is {cert_acc_mp}.  h_min is {hmin_mp}"
    print(string)
    do_log(filename, string)
    

    
@click.command()
@click.argument("config_path", type=click.Path(exists=True))
def main(config_path):
    with open(config_path, "r") as f:
        config = yaml.load(f, Loader=yaml.FullLoader)

    calculate_general(config)

if __name__ == "__main__":
    main()

Overwriting certify_gl.py


In [84]:
import numpy as np
np.linspace(1, 16, 2)

array([ 1., 16.])

In [87]:
r = lambda h: -np.log(2 - 2 * h)
h = np.linspace(0.51, 1, 101)

r(h)

/tmp/ipykernel_849/1519491582.py:1: RuntimeWarning: divide by zero encountered in log
  r = lambda h: -np.log(2 - 2 * h)


array([0.02020271, 0.03025304, 0.04040541, 0.05066191, 0.0610247 ,
       0.071496  , 0.08207811, 0.0927734 , 0.10358432, 0.11451339,
       0.12556322, 0.13673652, 0.14803608, 0.15946477, 0.1710256 ,
       0.18272164, 0.19455609, 0.20653229, 0.21865365, 0.23092374,
       0.24334626, 0.25592504, 0.26866407, 0.28156747, 0.29463955,
       0.30788478, 0.3213078 , 0.33491345, 0.34870677, 0.36269302,
       0.37687765, 0.39126639, 0.40586519, 0.42068027, 0.43571815,
       0.45098562, 0.46648981, 0.48223817, 0.49823851, 0.51449903,
       0.53102833, 0.54783545, 0.56492988, 0.58232163, 0.6000212 ,
       0.61803971, 0.63638885, 0.65508098, 0.67412917, 0.69354726,
       0.71334989, 0.7335526 , 0.75417188, 0.77522529, 0.7967315 ,
       0.8187104 , 0.84118326, 0.86417278, 0.88770328, 0.91180083,
       0.93649344, 0.96181125, 0.98778673, 1.01445498, 1.04185395,
       1.07002483, 1.09901237, 1.12886533, 1.15963699, 1.19138569,
       1.22417551, 1.25807706, 1.29316838, 1.32953603, 1.36727

In [15]:
import yaml
config_path = "configs/cb/cifar10.yaml"
with open(config_path, "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [32]:
for beta in config["betas_estimation"]:
    print(beta)

beta0
beta1


In [31]:
config["betas_estimation"]

{'beta0': {'left': 0.5, 'right': 1.5, 'db': 41},
 'beta1': {'left': -0.5, 'right': 0.5, 'db': 43}}

In [ ]:
    args.corrupt = config["attack"]
    args.noise_sd = config["noise_sd"]
    args.noise_dst = config["noise_dst"]
    args.partial_max = config["partial_max"]
    args.dataset = config["dataset"]
    args.base_classifier = config["base_classifier"]
    args.device = config["device"]
    args.arch = config["arch"]

In [ ]:
base_classifier = model
    
    dataset, _ = make_test_dataset(args)
    
    base_classifier.to(device)
    base_classifier.eval()

    corruptor = Corruption(args, co_type=args.corrupt,add_noise=args.add_noise,noise_sd=args.noise_sd,distribution=args.noise_dst)

    # create the smooothed classifier g
    smoothed_classifier = TSmooth(base_classifier, None, corruptor, get_num_classes(args.dataset),args.noise_dst,args.noise_sd, args.add_noise)

    # prepare output file
    filename = args.outfile+'_'+args.dataset+'_'+args.corrupt+'_'+str(args.noise_sd) +"_" +str(args.partial_max)
    f = open(filename+'_running', 'w')
    print("idx\tlabel\tpredict\tradius\tgood\tcorrect\ttime", file=f, flush=True)
    print("idx\tlabel\tpredict\tradius\tgood\tcorrect\ttime")

    tot, tot_good, tot_correct = 0, 0, 0

    # for gaussian smooth
    attack_radius = args.partial_max
    for i in tqdm(range(len(dataset))):
        (x, label) = dataset[i]

        before_time = time()
        x = x.to(device)
        prediction, radius = smoothed_classifier.certify(x, args.N0, args.N, args.alpha, args.batch)


        correct = (prediction == label).item()
        cond1 = radius * args.noise_sd > args.partial_max
        good = (radius * args.noise_sd > args.partial_max)&correct

        tot, tot_good, tot_correct = tot+1, tot_good+good, tot_correct+correct
        after_time = time()
        time_elapsed = str(datetime.timedelta(seconds=(after_time - before_time)))
        print("{}\t{}\t{}\t{:.5f}\t{}\t{}\t{}".format(i, label, prediction, radius, good, correct, time_elapsed), file=f, flush=True)
        print("{}\t{}\t{}\t{:.5f}\t{}\t{}\t{}".format(i, label, prediction, radius, good, correct, time_elapsed))

    f.close()

    print("Total {} Certified {} Certified Acc {} Test Acc {}".format(tot, tot_good, tot_good/tot, tot_correct/tot))

    f = open(filename+'_total_result', 'w')
    print("Total {} Certified {} Certified Acc {} Test Acc {}".format(tot, tot_good, tot_good/tot, tot_correct/tot), file=f, flush=True)
    f.close()

In [36]:
import datetime


In [40]:
datetime.time()

datetime.time(0, 0)

In [62]:
"{:%Y_%m_%d_%H_%M_%S}".format(datetime.now())

'2024_02_01_20_07_59'

In [61]:
from datetime import datetime, date

print("{:%Y_%m_%d_%h_%m}".format(datetime.now()))

2024_02_01_Feb_02


In [43]:
datetime.now()

datetime.datetime(2024, 2, 1, 19, 36, 59, 701883)

In [53]:
sigmas = [1, 2, 3, 4, 5.5]

sigmas_values = "b_{}_c_{}_tr_{}_gamma_{}_blur_{}".format(*sigmas)

In [54]:
sigmas_values

'b_1_c_2_tr_3_gamma_4_blur_5.5'

In [73]:
from datetime import datetime
str(datetime.now())
"{:%Y_%m_%d_%H_%M_%S}".format(datetime.now())

'2024_02_01_20_35_05'

In [76]:
config["sigmas"].values()

dict_values([0.3, 0.3, None, None, None])

In [79]:
sigmas_values = "b_{}_c_{}_tr_{}_gamma_{}_blur_{}".format(*config["sigmas"].values())

In [80]:
sigmas_values

'b_0.3_c_0.3_tr_None_gamma_None_blur_None'